In [25]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import glob
import re
import math
from time import time
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard, CSVLogger#, LearningRateScheduler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
#from sklearn.datasets import make_multilabel_classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
# import tensorflow_hub as hub
from sklearn.metrics import mean_absolute_error,mean_squared_error
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
from model8 import model
import argparse
import gc
import random

In [2]:
# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
weight_dir = '/content/drive/MyDrive/weights_C3D_sports1M_tf.h5'

In [4]:
def calculate_mean_std(x, channels_first=False, verbose=0):
    """
    Calculates channel-wise mean and std
    
    Parameters
    ----------
    x : array
        Array representing a collection of images (frames) or
        collection of collections of images (frames) - namely video
    channels_first : bool, optional
        Leave False, by default False
    verbose : int, optional
        1-prints out details, 0-silent mode, by default 0
    
    Returns
    -------
    array of shape [2, num_channels]
        Array with per channel mean and std for all the frames
    """
    ndim = x.ndim
    assert ndim in [5,4]
    assert channels_first == False
    all_mean = []
    all_std = []    
    num_channels = x.shape[-1]
    
    for c in range(0, num_channels):
        if ndim ==5: # videos
            mean = x[:,:,:,:,c].mean()
            std = x[:,:,:,:,c].std()
        elif ndim ==4: # images rgb or grayscale
            mean = x[:,:,:,c].mean()
            std = x[:,:,:,c].std()
        if verbose:
            print("Channel %s mean before: %s" % (c, mean))   
            print("Channel %s std before: %s" % (c, std))
            
        all_mean.append(mean)
        all_std.append(std)
    
    return np.stack((all_mean, all_std))


def preprocess_input(x, mean_std, divide_std=False, channels_first=False, verbose=0):
    """
    Channel-wise substraction of mean from the input and optional division by std
    
    Parameters
    ----------
    x : array
        Input array of images (frames) or videos
    mean_std : array
        Array of shape [2, num_channels] with per-channel mean and std
    divide_std : bool, optional
        Add division by std or not, by default False
    channels_first : bool, optional
        Leave False, otherwise not implemented, by default False
    verbose : int, optional
        1-prints out details, 0-silent mode, by default 0
    
    Returns
    -------
    array
        Returns input array after applying preprocessing steps
    """
    x = np.asarray(x, dtype=np.float32)    
    ndim = x.ndim
    assert ndim in [5,4]
    assert channels_first == False
    num_channels = x.shape[-1]
    
    for c in range(0, num_channels):  
        if ndim ==5: # videos
            x[:,:,:,:,c] -= mean_std[0][c]
            if divide_std:
                x[:,:,:,:,c] /= mean_std[1][c]
            if verbose:
                print("Channel %s mean after preprocessing: %s" % (c, x[:,:,:,:,c].mean()))    
                print("Channel %s std after preprocessing: %s" % (c, x[:,:,:,:,c].std()))
        elif ndim ==4: # images rgb or grayscale
            x[:,:,:,c] -= mean_std[0][c]
            if divide_std:
                x[:,:,:,c] /= mean_std[1][c]   
            if verbose:        
                print("Channel %s mean after preprocessing: %s" % (c, x[:,:,:,c].mean()))    
                print("Channel %s std after preprocessing: %s" % (c, x[:,:,:,c].std()))            
    return x


In [5]:
files = glob.glob('/content/drive/MyDrive/training_arr/*.avi')
print(str(files[5]))

training_labels = []
training_files = []

/content/drive/MyDrive/training_arr/television10367_clipped.avi


In [6]:
for file in files:
    label = re.findall('[A-Za-z]+[0-9]',str(file))[0][:-1]
    training_labels.append(label)
    training_files.append(str(file))

In [7]:
training_data = pd.DataFrame({'filename':training_files,'training_labels':training_labels})
print(training_data)

print(training_data)
label_encoder = LabelEncoder().fit_transform(training_data['training_labels'])
training_data['encoded_labels'] = label_encoder
print(training_data)
training_data.to_csv('training_words.csv')

                                               filename training_labels
0     /content/drive/MyDrive/training_arr/teacher778...         teacher
1     /content/drive/MyDrive/training_arr/teacher974...         teacher
2     /content/drive/MyDrive/training_arr/teacher976...         teacher
3     /content/drive/MyDrive/training_arr/teacher933...         teacher
4     /content/drive/MyDrive/training_arr/teacher974...         teacher
...                                                 ...             ...
5328  /content/drive/MyDrive/training_arr/beautiful2...       beautiful
5329  /content/drive/MyDrive/training_arr/beautiful2...       beautiful
5330  /content/drive/MyDrive/training_arr/beautiful2...       beautiful
5331  /content/drive/MyDrive/training_arr/beautiful8...       beautiful
5332  /content/drive/MyDrive/training_arr/beautiful7...       beautiful

[5333 rows x 2 columns]
                                               filename training_labels
0     /content/drive/MyDrive/training_a

In [8]:
videos_data = []


In [9]:
# i=1

# for file in training_data['filename']:
    
    
#     resize=(112, 112)
#     if i%100 == 0:
#       print(i)
#       print(str(file))

#     cap = cv2.VideoCapture(str(file))
#     ret = True
#     frames=[]

#     while ret == True:
#         ret,frame = cap.read()
#         if ret == True:
#             frame = cv2.resize(frame,resize)
#             frames.append(frame)
        
#     video = np.stack(frames,axis=0)
#     frames,length,width,channels = video.shape


#     video = video[list(np.linspace(0,frames-1,16,dtype=int))]
    
#     mean_std = calculate_mean_std(video, channels_first=False, verbose=0)
#     video = preprocess_input(video, mean_std, divide_std=False, channels_first=False, verbose=0)
#     videos_data.append(video)
#     cap.release()
#     i += 1
    

# cv2.destroyAllWindows()

In [10]:
def gen_video_prep(file_loc, size):

    resize = size

    cap = cv2.VideoCapture(str(file_loc))
    ret = True
      
    frames=[]

    while ret == True:
        ret, frame = cap.read()
        if ret == True:
            frame = cv2.resize(frame,resize)
            num = np.random.randint(0,100)
            if num < 20:
                frame = np.flip(frame,axis=1)
            frames.append(frame)
          
    video = np.stack(frames,axis=0)
    frames, channels = video.shape[0], video.shape[3]

    frame_total = 16
    if frames >= frame_total:
        frame_list = list(range(0,frames))
        random.shuffle(frame_list)
        frame_list = frame_list[:16]
        frame_list.sort(reverse = False)
    else:
        frame_list = list(np.linspace(0,frames-1,16,dtype=int))

    #video = video[list(np.linspace(0,frames-1,16,dtype=int))]
    video = video[frame_list]

    mean_std = calculate_mean_std(video, channels_first=False, verbose=0)

    video = preprocess_input(video, mean_std, divide_std=False, channels_first=False, verbose=0)

    cap.release()
      

    cv2.destroyAllWindows()
  
    return video

In [ ]:
def val_preprocess(file_loc, size):

    resize = size

    cap = cv2.VideoCapture(str(file_loc))
    ret = True
      
    frames=[]

    while ret == True:
        ret, frame = cap.read()
        if ret == True:
            frame = cv2.resize(frame,resize)
            frames.append(frame)
          
    video = np.stack(frames,axis=0)
    frames, channels = video.shape[0], video.shape[3]

    frame_total = 16
    if frames >= frame_total:
        frame_list = list(range(0,frames))
        random.shuffle(frame_list)
        frame_list = frame_list[:16]
        frame_list.sort(reverse = False)
    else:
        frame_list = list(np.linspace(0,frames-1,16,dtype=int))

    #video = video[list(np.linspace(0,frames-1,16,dtype=int))]
    video = video[frame_list]

    mean_std = calculate_mean_std(video, channels_first=False, verbose=0)

    video = preprocess_input(video, mean_std, divide_std=False, channels_first=False, verbose=0)

    cap.release()
      

    cv2.destroyAllWindows()
  
    return video

In [12]:
# for step in range(num_steps):
#     # Pick an offset within the training data, which has been randomized.
#     # Note: we could use better randomization across epochs.
#     offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

#     # Generate a minibatch.
#     batch_data = train_dataset[offset:(offset + batch_size), :]
#     batch_labels = train_labels[offset:(offset + batch_size), :]

#     # Prepare a dictionary telling the session where to feed the minibatch.
#     # The key of the dictionary is the placeholder node of the graph to be fed,
#     # and the value is the numpy array to feed to it.
#     feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

class My_Custom_Generator(tf.keras.utils.Sequence) :
  
  def __init__(self, video_filenames, labels, batch_size) :
      self.video_filenames = video_filenames
      self.labels = labels
      self.batch_size = batch_size
    
    
  def __len__(self) :
      return (np.ceil(len(self.video_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
      batch_x = self.video_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
      batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
      return np.array([
              gen_video_prep(file_name, (112, 112)) for file_name in batch_x]), np.array(batch_y)

In [13]:
# with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
#   conv_model = model(weight_dir, trainable = True, freeze_layer = 0)
#   conv_model = conv_model.retrainable_model((3, 3, 3), (16, 112, 112, 3))
#   conv_model.summary()
#   conv_model.compile(optimizer='adam',
#                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#                 metrics=['sparse_categorical_accuracy'])



In [14]:
# conv_model = conv_model.retrainable_model((3, 3, 3), (16, 112, 112, 3))
# conv_model.summary()



In [15]:
conv_model = model(weight_dir, trainable = True, freeze_layer = 0)
conv_model = conv_model.retrainable_model((3, 3, 3), (16, 112, 112, 3))
conv_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
Pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
Conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
Pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
Conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
Conv3b (Conv3D)              (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
Pool3 (MaxPooling3D)         (None, 4, 14, 14, 256)    0

In [16]:
y = np.asarray(training_data['encoded_labels'].values)


In [17]:
print(y.shape)
y = to_categorical(y)
print(y.shape)



(5333,)
(5333, 298)


In [18]:
gen = My_Custom_Generator(training_data['filename'].tolist(), y, 32)

In [19]:
# tf.config.run_functions_eagerly(True)

In [30]:
model_path = '/content/drive/MyDrive/model'
log_path = '/content/drive/MyDrive/model_log' 
model_name = 'weights.best.{epoch:03d}-{accuracy:.4f}.hdf5'
tb_path = '/content/drive/MyDrive/tb_logs'

In [27]:
# def scheduler(epoch, lr):

#     if epoch < 50:
#         return lr
#     elif epoch == 50:
#         return lr*0.2
#     elif epoch == 75:
#         return lr*0.5
#     elif epoch == 100:
#         return lr*0.2
#     elif epoch == 125:
#         return lr*0.5
#     elif epoch == 150:
#         return lr*0.2

In [34]:
callbacks = [
    ModelCheckpoint(
        filepath = os.path.join(model_path, model_name),
        monitor = 'accuracy', 
        save_best_only = True, 
        mode = 'max'
        ),

    CSVLogger(
        filename=os.path.join(log_path, 'log.csv'), 
        separator = ',', 
        append = True
        ),

    EarlyStopping(
        monitor = 'loss',
        patience = 10
        ),

    TensorBoard(
        log_dir = tb_path,
        histogram_freq = 10,
        write_graph = True,
        write_images = True,
        write_steps_per_second = False,
        update_freq = 'epoch',
        profile_batch = 0,
        embeddings_freq = 0,
        embeddings_metadata = None
        )#,

    # LearningRateScheduler(
    #     schedule = scheduler
    #     ),
    ]



In [36]:
K.set_value(conv_model.optimizer.lr, 5e-4) # 5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4

In [ ]:
start = time()
conv_model.fit(gen,epochs=400,batch_size=32,callbacks=callbacks)
print(time()-start)

Epoch 1/400
167/167 [==============================] - 137s 818ms/step - loss: 2.2870 - accuracy: 0.4894
Epoch 2/400
167/167 [==============================] - 133s 797ms/step - loss: 2.2400 - accuracy: 0.4956
Epoch 3/400
167/167 [==============================] - 134s 801ms/step - loss: 2.1743 - accuracy: 0.5097
Epoch 4/400
167/167 [==============================] - 136s 812ms/step - loss: 2.1147 - accuracy: 0.5263
Epoch 5/400
  5/167 [..............................] - ETA: 2:11 - loss: 2.1398 - accuracy: 0.5188